In [1]:
open("log/live_log.log","w")

mode = "Live" # Backtesting/Live/Algo

%run "KiteConnect_Library.ipynb"   
warnings.filterwarnings('ignore')



In [2]:
%run "KiteConnect_Strategies_v2.ipynb" 

In [3]:
logger.setLevel(logging.DEBUG)

gc.collect()

7

%%javascript
IPython.notebook.kernel.execute("URL = '" + window.location + "'");

In [4]:
portfolio = pd.DataFrame(downloadlist.tolist())
portfolio = pd.DataFrame(['BHARTIARTL','TCS'])

portfolioToken = portfolio[0].apply(EQTOKEN).tolist()

In [5]:
portfolio[0].iloc[-1]

'TCS'

In [6]:
papertrade = False

In [7]:
if papertrade:
    x = portfolio[0].iloc[-1]
    gc.collect()
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist.tolist())    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate)+"%")

In [8]:
if mode == "Live":
    kws.connect(threaded=True)

In [21]:
if mode == "Live":
    print(kws.is_connected())

False


In [20]:
if mode == "Live":
    kws.close()

In [ ]:
if mode == "Live":
    print(kws.is_connected())

In [23]:
kws.tradebook_df.loc['BHARTIARTL','symbol'].CLOSE

AttributeError: 'algoTrade' object has no attribute 'CLOSE'